### Data Preparation

In [104]:
import os
import json
import pandas as pd

# result.json contains flattened json (so list of dicts)
f = open(os.path.join(os.getcwd(), "..", "data_collection", "data_transformation_tests", "result.json"))
json_data = json.load(f)

# Convert list of dicts to dataframe
df = pd.DataFrame(json_data) 
df


,category.id,category.name,category.playlist.id,category.playlist.name,category.playlist.track.id,category.playlist.track.name,category.playlist.track.album.id,category.playlist.track.album.name,category.playlist.track.artist,category.playlist.track.feature.danceability,...,category.playlist.track.feature.loudness,category.playlist.track.feature.mode,category.playlist.track.feature.speechiness,category.playlist.track.feature.acousticness,category.playlist.track.feature.instrumentalness,category.playlist.track.feature.liveness,category.playlist.track.feature.valence,category.playlist.track.feature.tempo,category.playlist.track.feature.duration_ms,category.playlist.track.feature.time_signature
0,hiphop,Hip Hop,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,2BcMwX1MPV6ZHP4tUT9uq6,Knife Talk (with 21 Savage ft. Project Pat),3SpBlxme9WbeQdI9kx7KAV,Certified Lover Boy,"Drake, 21 Savage, Project Pat",0.849,...,-9.579,0,0.324,0.0635,0,0.0834,0.153,145.887,242966,4
1,hiphop,Hip Hop,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,3J4VKkUpLvomXJbn9ZAypO,Sharing Locations (feat. Lil Baby & Lil Durk),0q6qOUfC4ikWtv62KCnoip,Sharing Locations (feat. Lil Baby & Lil Durk),"Meek Mill, Lil Durk, Lil Baby",0.681,...,-5.585,1,0.0385,0.00383,0,0.139,0.183,151.951,161053,4
2,hiphop,Hip Hop,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,7Bpx2vsWfQFBACRz4h3IqH,family ties (with Kendrick Lamar),3HqmX8hGcbbQZODgayNEYx,family ties (with Kendrick Lamar),"Baby Keem, Kendrick Lamar",0.711,...,-5.453,1,0.329,0.00575,0,0.231,0.144,134.14,252070,4
3,hiphop,Hip Hop,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,52nTQQahs46s1iHFQ3Wv5Q,Pissed Me Off,70m1QwzUoogjk2DDq1hA3b,Pissed Me Off,Lil Durk,0.723,...,-10.707,0,0.485,0.00311,1.26e-06,0.115,0.223,155.967,123077,4
4,hiphop,Hip Hop,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,4KDNRh9Oor80z3XIxdWlui,Bubbly (with Drake & Travis Scott),7IKkHmEk4It4cRdOYanyvW,Punk,"Young Thug, Drake, Travis Scott",0.91,...,-7.572,0,0.257,0.0536,0,0.127,0.599,129.011,165067,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13755,rock,Rock,37i9dQZF1DX9aVADS5TZr5,Rock Your Rights,0gj1rE7vQafcByX2nwXy3W,So We Can Stay Alive,0EBQyjl6kq9iPYMWM7eLWN,Strange Little Birds,Garbage,0.247,...,-5.816,1,0.0411,0.000213,0.744,0.136,0.101,101.851,361373,4
13756,rock,Rock,37i9dQZF1DX9aVADS5TZr5,Rock Your Rights,5u8Dwz4h41DDOKfeFfexMR,Invisible Sun,5jkwdY6jS1Hzi8epr6HW7h,Ghost In The Machine (Remastered 2003),The Police,0.573,...,-10.927,1,0.031,0.425,0.267,0.0934,0.149,120.308,224200,4
13757,rock,Rock,37i9dQZF1DX9aVADS5TZr5,Rock Your Rights,12w7q5JN7tf22TzQtQiiBN,Bullet The Blue Sky,5y6wlw1LnqFnQFruMeiwGU,The Joshua Tree (Super Deluxe),U2,0.364,...,-10.24,1,0.0441,0.00735,0.399,0.133,0.451,152.509,271547,4
13758,rock,Rock,37i9dQZF1DX9aVADS5TZr5,Rock Your Rights,4CKITKW8alxoGiHu9792PO,The Wheel (Is Turning Now),6RaFspd5Yw90HK0CBBDquB,Get To Heaven (Deluxe Version),Everything Everything,0.37,...,-8.359,1,0.0532,0.00553,0.0799,0.0773,0.31,133.035,328200,4


In [105]:
# Drop unneeded columns
columns_to_drop = [
    "category.name",
    "category.playlist.id",
    "category.playlist.name",
    "category.playlist.track.id",
    "category.playlist.track.name",
    "category.playlist.track.album.id",
    "category.playlist.track.album.name",
    "category.playlist.track.artist"
    ]
df = df.drop(columns=columns_to_drop)

df = df.rename(columns={
    "category.id": "category",
    "category.playlist.track.feature.danceability": "feature_danceability",
    "category.playlist.track.feature.energy": "feature_energy",
    "category.playlist.track.feature.key": "feature_key",
    "category.playlist.track.feature.loudness": "feature_loudness",
    "category.playlist.track.feature.mode": "feature_mode",
    "category.playlist.track.feature.speechiness": "feature_speechiness",
    "category.playlist.track.feature.acousticness": "feature_acousticness",
    "category.playlist.track.feature.instrumentalness": "feature_instrumentalness",
    "category.playlist.track.feature.liveness": "feature_liveness",
    "category.playlist.track.feature.valence": "feature_valence",
    "category.playlist.track.feature.tempo": "feature_tempo",
    "category.playlist.track.feature.duration_ms": "feature_duration_ms",
    "category.playlist.track.feature.time_signature": "feature_time_signature"
})
df

,category,feature_danceability,feature_energy,feature_key,feature_loudness,feature_mode,feature_speechiness,feature_acousticness,feature_instrumentalness,feature_liveness,feature_valence,feature_tempo,feature_duration_ms,feature_time_signature
0,hiphop,0.849,0.424,5,-9.579,0,0.324,0.0635,0,0.0834,0.153,145.887,242966,4
1,hiphop,0.681,0.63,1,-5.585,1,0.0385,0.00383,0,0.139,0.183,151.951,161053,4
2,hiphop,0.711,0.611,1,-5.453,1,0.329,0.00575,0,0.231,0.144,134.14,252070,4
3,hiphop,0.723,0.516,11,-10.707,0,0.485,0.00311,1.26e-06,0.115,0.223,155.967,123077,4
4,hiphop,0.91,0.585,11,-7.572,0,0.257,0.0536,0,0.127,0.599,129.011,165067,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13755,rock,0.247,0.722,0,-5.816,1,0.0411,0.000213,0.744,0.136,0.101,101.851,361373,4
13756,rock,0.573,0.396,10,-10.927,1,0.031,0.425,0.267,0.0934,0.149,120.308,224200,4
13757,rock,0.364,0.638,8,-10.24,1,0.0441,0.00735,0.399,0.133,0.451,152.509,271547,4
13758,rock,0.37,0.72,7,-8.359,1,0.0532,0.00553,0.0799,0.0773,0.31,133.035,328200,4


In [106]:
df.dtypes

category                    object
feature_danceability        object
feature_energy              object
feature_key                 object
feature_loudness            object
feature_mode                object
feature_speechiness         object
feature_acousticness        object
feature_instrumentalness    object
feature_liveness            object
feature_valence             object
feature_tempo               object
feature_duration_ms         object
feature_time_signature      object
dtype: object

In [107]:
# sklearn takes the features and labels as seperate lists
# df needs to be split
def encode_target(df, target_column):

    df_mod = df.copy()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod)

df_target = encode_target(df, "category")

# Target to category mapping
df_target[["target", "category"]].head()

,target,category
0,0,hiphop
1,0,hiphop
2,0,hiphop
3,0,hiphop
4,0,hiphop


In [108]:
# Possible values for category
targets = df_target["category"].unique()
targets

array(['hiphop', 'rock'], dtype=object)

### Train model

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Split test and training data

train, test = train_test_split(df_target, test_size=0.2, random_state=45, shuffle=True)
print("Number of rows in train: ", train.shape[0])
print("Number of rows in test: ", test.shape[0])

Number of rows in train:  11008
Number of rows in test:  2752


In [110]:
# List of feature names

features = list(train.columns[1:14])
features

['feature_danceability',
 'feature_energy',
 'feature_key',
 'feature_loudness',
 'feature_mode',
 'feature_speechiness',
 'feature_acousticness',
 'feature_instrumentalness',
 'feature_liveness',
 'feature_valence',
 'feature_tempo',
 'feature_duration_ms',
 'feature_time_signature']

In [111]:
# Y is list containing all target values
Y = train["target"]
# X is dataframe containing all feature columns
X = train[features]

# Create decision tree classifier object
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)

# Fit model
dt.fit(X, Y)

DecisionTreeClassifier(min_samples_split=20, random_state=99)

In [112]:
# Use model on test data, this generates a list containing the predicted values for "target" 
prediction = dt.predict(test[features])
prediction

array([0, 0, 1, ..., 0, 0, 1])

In [113]:
# Calculate accuracy score using the actual and predicted values for "target"

accuracy_score(prediction, test["target"])

0.9364098837209303